In [ ]:
import sys
sys.path.append("..")
import numpy as np 
import torch
from relie.utils.so3_tools import so3_hat, so3_vee, so3_exp, so3_log
from relie.utils.se3_tools import se3_hat, se3_vee, se3_exp, se3_log, se3_inv

In [ ]:
def det_jac_so3(x):
    theta = x.norm(2,-1)
    return 2*(1 - torch.cos(theta))/(theta**2)

def so3_inv(el):
    return el.transpose(-2,-1)
    

In [ ]:
def compute_approx_jacobian(points, vee, exp, log, inv, eps = 0.01):
    points = points.unsqueeze(-2)
    dim = points.shape[-1]
    basis = torch.eye(dim, dtype=torch.float64).unsqueeze(0)
    group_delta = exp(basis*eps + points)
    points_inv = inv(exp(points))
    normal_coord = vee(log(points_inv@group_delta))
    print(normal_coord)
    estimated_det_jac = np.linalg.det(normal_coord.numpy())/((eps)**dim)
    return torch.tensor(estimated_det_jac)

### SO(3)

In [ ]:
def approximate_so3_jacobian(points, eps = 0.01):
    return compute_approx_jacobian(points, so3_vee, so3_exp, so3_log, so3_inv, eps)

In [ ]:
dim = 3
eps = 0.005

center = torch.tensor(np.random.normal(0,1,(10,dim)),dtype = torch.float64)
estimated_det_jac = approximate_so3_jacobian(center, eps=eps)
print(estimated_det_jac - det_jac_so3(center))

### SE(3)

In [ ]:
def approximate_se3_jacobian(points, eps = 0.01):
    return compute_approx_jacobian(points, se3_vee, se3_exp, se3_log, se3_inv, eps)

In [ ]:
dim = 6
eps = 1

center = torch.tensor(np.random.normal(0,1,(10,dim)),dtype = torch.float64)
estimated_det_jac = approximate_se3_jacobian(center, eps=eps)

In [ ]:
theta = torch.tensor(np.random.normal(0,1,(10,dim)),dtype = torch.float64)
((1 - theta*torch.sin(theta))/(2-2*torch.cos(theta)))/theta**2